In [1]:
import pickle
import math
import pandas as pd
import numpy as np
from numpy import array
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

Using TensorFlow backend.


In [2]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename,'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

In [3]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
def corpus_to_tokens_to_sequences(corpus,sequence_length=50):
    tokens = corpus.split()
    # organize into sequences of tokens
    length = sequence_length + 1
    sequences = list()
    for i in range(length, len(tokens)):
        # select sequence of tokens
        seq = tokens[i-length:i]
        # convert into a line
        line = ' '.join(seq)
        # store
        sequences.append(line)
    print('Total Sequences: %d' % len(sequences))
    return sequences

In [5]:
# Importing Torah
clean_torah = load_doc('clean_torah.txt')

In [6]:
# saving torahament sequences to txt file
torah_sequences = corpus_to_tokens_to_sequences(clean_torah)
out_filename = 'torah_sequences.txt'
save_doc(torah_sequences, out_filename)

Total Sequences: 150182


### Torah Text Generator

In [7]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(torah_sequences)
sequences = tokenizer.texts_to_sequences(torah_sequences)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
 
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
 
# define model
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_length),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dense(vocab_size, activation='softmax')
])

print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            244200    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 4884)              493284    
Total params: 888,384
Trainable params: 888,384
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/100

150182/150182 [==============================] - 150s 996us/sample - loss: 2.1512 - acc: 0.4847
Epoch 63/100
150182/150182 [==============================] - 169s 1ms/sample - loss: 2.0986 - acc: 0.4957
Epoch 64/100
150182/150182 [==============================] - 168s 1ms/sample - loss: 2.0660 - acc: 0.5022
Epoch 65/100
150182/150182 [==============================] - 169s 1ms/sample - loss: 2.0167 - acc: 0.5118
Epoch 66/100
150182/150182 [==============================] - 156s 1ms/sample - loss: 1.9774 - acc: 0.5206
Epoch 67/100
150182/150182 [==============================] - 187s 1ms/sample - loss: 1.9526 - acc: 0.5249
Epoch 68/100
150182/150182 [==============================] - 175s 1ms/sample - loss: 1.9335 - acc: 0.5281
Epoch 69/100
150182/150182 [==============================] - 176s 1ms/sample - loss: 1.9101 - acc: 0.5334
Epoch 70/100
150182/150182 [==============================] - 190s 1ms/sample - loss: 1.8903 - acc: 0.5373
Epoch 71/100
150182/150182 [====================

KeyboardInterrupt: 

In [8]:
# # # save the model to file
# model.save('torah_model.hdf5')
# # save the tokenizer
# pickle.dump(tokenizer, open('torah_tokenizer.pkl', 'wb'))